# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [5]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [8]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [9]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog / news', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
7 days ago
•
286k
•
2.74k
zai-org/GLM-4.6V-Flash
Updated
6 days ago
•
84.2k
•
436
microsoft/VibeVoice-Realtime-0.5B
Updated
3 days ago
•
143k
•
850
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
about 23 hours ago
•
21.4k
•
349
zai-org/GLM-4.6V
Updated
6 days ago
•
3.68k
•
308
Browse 1M+ models
Spaces
Running
on
Zero
582
Z Image Turbo
🖼
582
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.38k
Z Image Turbo
🏃
1.38k
Generate images from text prompts
Running
on
Zero
MCP
145

In [22]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more snarky brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates an critical, discerning and snarky brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a brochure available in Spanish :

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates short brochure about the company about the company for prospective customers, investors and recruits in Spanish.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Here is Brochure System Prompt for an advocating brochure for a non-profit that is close to my heart, written for English and Malay readers.

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a non-profit organisation website
and creates clear advocating and inspiring brochure about the organisation about the company for prospective volunteers, fundraisers and recruits in English and Bahasa Melayu.
Keep the flow and language engaging and entertaining for English and flowery and poetic for Bahasa Melayu.
Respond in markdown without code blocks.
Include details of organistation vision, key initatives, culture, key contributors, amazing volunteers and careers/jobs if you have the information.
"""

In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n7 days ago\n•\n286k\n•\n2.74k\nzai-org/GLM-4.6V-Flash\nUpdated\n6 days ago\n•\n84.2k\n•\n436\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n3 days ago\n•\n143k\n•\n850\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\nabout 23 hours ago\n•\n21.4k\n•\n349\nzai-org/GLM-4.6V\nUpdated\n6 days ago\n•\n3.68k\n•\n308\nBrowse 1M+ models\nSpaces\nRunning\non\nZ

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is the premier collaboration platform for the machine learning (ML) community, dedicated to building an open and ethical AI future. It serves as a central hub where ML engineers, scientists, and end users can create, share, discover, and experiment with open-source machine learning models, datasets, and applications.

Harnessing a fast-growing global community, Hugging Face powers over 1 million models, 250,000+ datasets, and 400,000+ applications across text, image, video, audio, and 3D modalities. With cutting-edge open source tools and a talented science team exploring the frontier of AI, Hugging Face is at the heart of the AI revolution.

---

## What We Offer

- **Models & Datasets Hub**: Access, host, and collaborate on millions of publicly available models and datasets with extensive search and version control capabilities.
- **Spaces**: Deploy AI-powered applications in the cloud instantly to showcase your work or prototype innovative solutions.
- **Open Source Stack**: Utilize a rich ecosystem of libraries and tools designed to speed up ML development and deployment.
- **Community & Collaboration**: Engage with a vibrant community of ML practitioners through forums, discussions, and collaborative projects.
- **Learn & Share**: Build and showcase your ML portfolio, grow your skills, and contribute to AI advancements.
- **Enterprise Solutions**: Scalable AI platform with enterprise-grade security, dedicated support, and customizable features tailored to organizational needs.

---

## Enterprise Platform Features

Hugging Face’s **Team & Enterprise Hub** empowers organizations with:

- **Secure Single Sign-On (SSO)**
- **Data region selection and comprehensive audit logs**
- **Granular access and token management**
- **Organization-wide analytics and usage monitoring dashboards**
- **Advanced compute options including ZeroGPU and quota boosts**
- **Enhanced storage and private dataset management**
- **Flexible subscription models starting at $20/user/month**

Built for scalability and control, this platform meets enterprise-level governance, compliance, and security standards while accelerating AI innovation.

---

## Company Culture

At Hugging Face, the culture is rooted in **open collaboration, transparency, and ethical AI innovation**. The company supports a diverse and passionate community focused on:

- Building open-source tools that democratize AI technology.
- Promoting responsible and inclusive AI development.
- Empowering individuals and teams across the globe to learn, contribute, and grow in the ML domain.

Hugging Face fosters a creative and inclusive work environment where innovation flourishes through teamwork and shared goals.

---

## Careers at Hugging Face

Join a visionary company at the forefront of the AI movement. Hugging Face offers dynamic career opportunities for:

- Machine Learning Engineers and Scientists
- Software Developers and Platform Engineers
- Research Scientists exploring AI frontiers
- Community Managers and Educators
- Enterprise Sales and Customer Success professionals

Working at Hugging Face means contributing to impactful open source projects, collaborating with world-class experts, and helping shape the future of AI technology.

Explore open positions on their Careers page and become part of a thriving, future-focused team.

---

## Connect and Explore

- Visit [huggingface.co](https://huggingface.co) to explore models, datasets, and AI apps.
- Engage with the community on GitHub, Twitter, LinkedIn, and Discord.
- Access comprehensive documentation, blogs, and forums to stay updated.
- Discover premium enterprise solutions tailored for your organizational AI journey.

---

## Brand Identity

Hugging Face’s vibrant brand reflects its commitment to openness, energy, and innovation, with signature colors including vibrant yellow (#FFD21E) and orange (#FF9D00), and a friendly, approachable logo symbolizing connection and collaboration.

---

**Join Hugging Face — Where the AI Community Builds the Future Together.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It is a collaborative platform where the machine learning community creates, shares, and improves models, datasets, and applications across modalities including text, image, video, audio, and 3D.

With over **1 million models**, **250,000+ datasets**, and **400,000+ applications**, Hugging Face provides an extensive ecosystem empowering researchers, developers, and enterprises alike to accelerate AI innovation. 

---

## What We Offer

### Core Platform Features

- **Models:** Access and contribute to a vast repository of models ranging from state-of-the-art natural language processing to computer vision.
- **Datasets:** Discover and share datasets optimizing AI training and testing performance.
- **Spaces:** Run and showcase machine learning applications easily on scalable infrastructure with zero infrastructure management.
- **Community:** Engage with a global community of AI practitioners, share your work, build your portfolio, and collaborate on projects.

### Enterprise Solutions

For organizations seeking to build AI at scale, Hugging Face offers:

- **Team & Enterprise Plans** starting at $20/user/month with flexible contractual options.
- **Enterprise-grade security:** Single Sign-On (SSO), comprehensive audit logs, granular access controls, and organization-wide security policies.
- **Private Storage & Dataset Viewer:** Extensive private storage options with user-specific quotas and tools for private dataset collaboration.
- **Advanced Compute:** Access scalable compute resources including ZeroGPU quota boosts for faster model deployment.
- **Analytics & Billing:** Central dashboards to monitor repository usage, spending limits, and optimized billing.
- **Priority Dedicated Support:** Expert Hugging Face support to maximize your platform utilization.

---

## Pricing Overview

- **Free Tier:** Access to public models, datasets, and applications with community support.
- **Pro Account ($9/month):** Enhanced personal experience with increased private storage, inference credits, priority queues, and advanced hosting capabilities.
- **Team ($20/user/month):** Instant set-up for growing teams with additional security and collaboration features.
- **Enterprise:** Customized plans for organizations needing comprehensive AI infrastructure and compliance.

---

## Our Customers and Community

Hugging Face serves a diverse set of users — from independent researchers and open-source enthusiasts to enterprise teams across industries such as tech, healthcare, automotive, and finance. Leading AI organizations trust Hugging Face to accelerate their machine learning projects.

The platform thrives on community contributions and collaboration, making it a vibrant hub where people exchange ideas, share models, datasets, and applications, collectively pushing the boundaries of AI.

---

## Company Culture

Hugging Face fosters an open, inclusive, and innovative environment driven by collaboration and shared progress in AI. The company is passionate about democratizing AI technology, empowering users of all skill levels, and nurturing a strong community spirit where ideas and knowledge flow freely.

---

## Careers

Join the team that’s shaping the future of artificial intelligence:

- Work alongside AI researchers, engineers, and community builders in a fast-paced, mission-driven environment.
- Engage in meaningful work that impacts millions of users worldwide.
- Access to the latest AI technologies and resources.
- Opportunities to contribute to open-source projects with global impact.
- Competitive benefits and investment in personal growth.

Interested candidates can explore open positions and apply through the Hugging Face website.

---

## Connect with Us

- **Website:** https://huggingface.co
- **Community:** Collaborate on models, datasets, and apps at our platform
- **Enterprise Sales:** Contact sales for tailored AI solutions and demos
- **Pricing & Plans:** Explore personal, team, and enterprise subscriptions

---

Build, share, and collaborate on AI with **Hugging Face** — the home of machine learning innovation.  
*The AI community building the future.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Changing the system prompt to the snarky version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Changing the system prompt to the Spanish version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [23]:
# Changing the system prompt to advocate for non-profit organisation that is close to my heart:

stream_brochure("Singapore Muslim Women Organisation", "https://ppis.sg/aboutus/")

Selecting relevant links for https://ppis.sg/aboutus/ by calling gpt-5-nano
Found 12 relevant links


# Singapore Muslim Women Organisation (PPIS)  
*Furthering Women Empowerment in Singapore*

---

## About Us

Founded in 1952, the **Singapore Muslim Women Organisation (PPIS)** has been a steadfast non-profit champion dedicated to serving women of all ages. With a deep commitment to helping women juggle their many roles in society, PPIS has grown into a beacon of support and empowerment across Singapore.

Our organisation runs three core community services:
- **Family Services**: Supporting families through challenges and transitions.
- **Student Care**: Providing holistic educational and emotional care for children.
- **Residential Facility**: A secular halfway house aiming to rehabilitate and reintegrate women into society.

Operating through **10 centres island-wide**, we deliver quality, all-encompassing programs for women and their families, addressing the unique needs at every life stage.

---

## Our Vision & Mission

**Vision:**  
*A Community of Successful Women and Thriving Families*  

**Mission:**  
*Inspiring Women • Strengthening Families*  

Madam Rahayu Mohamad, a visionary leader of PPIS, believes that modern leadership is not about commanding but about connecting with heart and empathy. Our organisation thrives on caring deeply for the people we serve — ensuring everyone feels valued, heard, and inspired.

---

## Key Initiatives

- **RED for Women & RED Space**: Dynamic platforms empowering women through networking, support, and personal growth.
- **Legal Consultation**: Accessible legal support tailored for women and families.
- **Family Service Centres**: Empowering families through counseling, workshops, and support groups.
- **FAM (PPIS)**: Support for remarriages and blended families.
- **Young Marriages Support**: Nurturing young couples for stronger family foundations.
- **Fostering Singapore**: Encouraging the nurturing of children in a safe and caring environment.
- **Children Services**: Including preschool and student care, facilitating developmental growth.
- **Rise Above Programme**: Building psycho-social resilience in women and families.

---

## Volunteer & Career Opportunities

Join **PPIS** if you wish to make a tangible difference in the lives of women and families. Whether as a passionate volunteer or a committed professional, your contribution will shape futures.

### Why Volunteer with PPIS?  
- Meaningful impact on diverse communities  
- Training and development opportunities  
- A supportive and empathetic culture  

### Career Paths Include:  
- Social Work and Counseling  
- Early Childhood Education  
- Administrative and Program Coordination  
- Community Outreach and Engagement  

---

## Culture & Values

At PPIS, we operate with **heart, empathy, and respect**. Our culture nurtures an environment where every voice counts, and every woman is celebrated for her strength and potential. We believe in **inclusive leadership** and continuous learning for sustainable community impact.

---

## Celebrate Our Amazing Contributors

Generations of dedicated volunteers and community workers have powered PPIS since 1952. Their selfless devotion fuels our ongoing success and growth — inspiring others to join the movement for women’s empowerment.

---

## Get Involved

Ready to make a difference?  
- **Volunteer**: Join our growing team of compassionate helpers.  
- **Fundraise**: Support our programs that change lives.  
- **Donate**: Ensure women and families receive the care they deserve.  

Discover more and connect with us today at:  
[ppis.org.sg](https://ppis.org.sg/)  

---

# **Together, We Empower Women, Strengthen Families, and Build Communities.**

---

# Organisasi Wanita Muslimah Singapura (PPIS)  
*Memperkasakan Wanita, Meningkatkan Keluarga di Singapura*

---

## Mengenai Kami  

Sejak tahun 1952, **PPIS** adalah sebuah organisasi bukan untung yang tekun berdedikasi untuk melayani wanita di semua peringkat umur. Berbekalkan semangat melestarikan peranan wanita dalam masyarakat, PPIS telah menjadi pemancar sokongan dan pemberdayaan yang kukuh di seluruh Singapura.

Kami mengendalikan tiga perkhidmatan utama:  
- **Perkhidmatan Keluarga**: Membantu keluarga menghadapi cabaran dan perubahan.  
- **Penjagaan Pelajar**: Memberi penjagaan menyeluruh kepada anak-anak dari segi pendidikan dan emosi.  
- **Fasiliti Penempatan Semula**: Rumah selingan sekular untuk wanita dalam program rehabilitasi komuniti.

Dengan **10 pusat di seluruh pulau**, kami menyediakan sokongan menyeluruh dan program pembangunan untuk wanita dan keluarga mereka sepanjang fasa kehidupan berbeza.

---

## Visi & Misi  

**Visi:**  
*Komuniti Wanita Yang Berjaya dan Keluarga Yang Makmur*  

**Misi:**  
*Mengilham Wanita • Memperkukuh Keluarga*  

Madam Rahayu Mohamad menegaskan kepimpinan PPIS bukan sekadar arahan dan kawalan, tetapi penuh dengan rasa empati dan hati yang ikhlas. Kami mempercayai, setiap insan perlu dihargai, didengari, dan diberikan inspirasi.

---

## Inisiatif Utama

- **RED for Women & RED Space**: Platform dinamik untuk memperkasakan wanita melalui jaringan dan pembangunan diri.  
- **Konsultasi Guaman**: Sokongan guaman yang mudah diakses untuk wanita dan keluarga.  
- **Pusat Perkhidmatan Keluarga**: Memberi kaunseling, bengkel, dan kumpulan sokongan keluarga.  
- **FAM (PPIS)**: Menyokong perkahwinan semula dan keluarga campuran.  
- **Sokongan Perkahwinan Muda**: Membina asas keluarga yang kukuh.  
- **Fostering Singapore**: Mendorong penjagaan penuh kasih sayang untuk anak-anak.  
- **Perkhidmatan Kanak-kanak**: Pra-sekolah dan penjagaan pelajar bagi perkembangan menyeluruh.  
- **Program Rise Above**: Membina ketahanan psikososial wanita dan keluarga.

---

## Peluang Sukarelawan & Kerjaya  

Bersama PPIS, anda boleh mengubah hidup wanita dan keluarga menjadi lebih bermakna. Sama ada sebagai sukarelawan penuh semangat atau profesional berdedikasi, sumbangan anda sangat berharga.

**Mengapa Sertai PPIS?**  
- Memberi impak bermakna kepada komuniti  
- Peluang latihan dan pembangunan  
- Budaya inklusif dan prihatin  

**Kerjaya Yang Ditawarkan:**  
- Kerja Sosial dan Kaunseling  
- Pendidikan Awal Kanak-Kanak  
- Pentadbiran dan Koordinasi Program  
- Perhubungan Komuniti & Penglibatan  

---

## Budaya & Nilai  

PPIS berakar dalam suasana penuh **hati, empati, dan hormat**. Kami meraikan kekuatan dan potensi setiap wanita, membina kepimpinan inklusif bagi kesan komuniti yang berpanjangan.

---

## Inspirasi Dari Para Penyumbang Hebat  

Sejak 1952, sukarelawan dan pekerja komuniti PPIS telah memberi sumbangan tanpa jemu. Semangat mereka yang murni terus menyemarakkan pembaharuan dan pertumbuhan organisasi.

---

## Sertailah Kami  

Bersedia menjadi agen perubahan?  
- **Sukarelawan**: Jadilah sebahagian daripada pasukan kami yang berdedikasi.  
- **Buat Derma**: Sokong program kami menyentuh kehidupan.  
- **Kutip Dana**: Bantu kami menyediakan penjagaan terbaik untuk wanita dan keluarga.  

Ketahui lebih lanjut di:  
[ppis.org.sg](https://ppis.org.sg/)  

---

# **Bersama, Kita Memberdaya Wanita, Memperkasa Keluarga, Membangun Komuniti.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>